In [76]:
import json
from fuzzywuzzy import fuzz
import psycopg2
import pyperclip
import tkinter as tk
from tkinter import messagebox
import sys
from decimal import Decimal

In [3]:
# Чтение json конфига
with open('config.json') as file:
    config = json.load(file)

pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

In [65]:
dupl_id_list = (113741935)
id_to_double_check = []

In [63]:
d_c_r_flag = True
dupl_id_to_search = dupl_id_list

while d_c_r_flag:
    dupl_cycle_cursor = conn.cursor()
    query = """
    select id, duplicate_id
    from av_test
    where duplicate_id = %s
    """ % (dupl_id_to_search)
    dupl_cycle_cursor.execute(query)
    d_c_r = dupl_cycle_cursor.fetchall()
    if d_c_r:
        for i in d_c_r:
            id_to_double_check.append(i[0])
        dupl_id_to_search = id_to_double_check[-1]
    else:
        d_c_r_flag = False

In [ ]:
search_id_list = []
dupl_id_list = []
id_to_add_list = []

dupl_cycle_cursor = conn.cursor()
query = """
select id, duplicate_id
from av_test
where duplicate_id = %s
""" % (dupl_id_to_search)
dupl_cycle_cursor.execute(query)
d_c_r = dupl_cycle_cursor.fetchall()

In [66]:
dupl_id_list = [113741935]  # Список с самым новым ID, для которого ищем дубликаты
processed_ids = set()  # Храним уже обработанные ID
all_duplicates = set(dupl_id_list)  # Храним все найденные дубликаты

while dupl_id_list:
    # Берем текущий ID для поиска
    current_id = dupl_id_list.pop(0)
    if current_id in processed_ids:
        continue  # Пропускаем, если ID уже обработан

    # Выполняем SQL-запрос для поиска всех объявлений, у которых duplicate_id = current_id
    with conn.cursor() as cursor:
        query = """
        SELECT id
        FROM av_test
        WHERE duplicate_id = %s
        """
        cursor.execute(query, (current_id,))
        results = cursor.fetchall()

    # Добавляем найденные ID в список для последующей обработки
    for row in results:
        new_id = row[0]
        if new_id not in processed_ids:  # Проверяем, чтобы не было повторов
            dupl_id_list.append(new_id)
            all_duplicates.add(new_id)

    # Помечаем текущий ID как обработанный
    processed_ids.add(current_id)

# Результат
print("Все дубликаты:", all_duplicates)

Все дубликаты: {Decimal('113741928'), Decimal('113741929'), Decimal('113741930'), Decimal('113741931'), Decimal('113741933'), Decimal('113741934'), 113741935, Decimal('110423743')}


In [ ]:
110423743, 113741928, 113741929, 113741930, 113741931, 113741933, 113741934, 113741935

In [77]:
def find_all_duplicates(initial_ids):
    """
    Функция для поиска всех связанных дубликатов.
    :param initial_ids: Список ID (dupl_id_list), с которых начинается поиск.
    :return: Список всех связанных ID.
    """
    dupl_id_list = list(initial_ids)  # Начальные ID для поиска
    processed_ids = set()  # Храним уже обработанные ID
    all_duplicates = set(dupl_id_list)  # Храним все найденные дубликаты

    while dupl_id_list:
        # Берем текущий ID для поиска
        current_id = dupl_id_list.pop(0)
        if current_id in processed_ids:
            continue  # Пропускаем, если ID уже обработан

        # Выполняем SQL-запрос для поиска всех объявлений, у которых duplicate_id = current_id
        with conn.cursor() as cursor:
            query = """
            SELECT id
            FROM av_test
            WHERE duplicate_id = %s
            """
            cursor.execute(query, (current_id,))
            results = cursor.fetchall()

        # Добавляем найденные ID в список для последующей обработки
        for row in results:
            new_id = row[0]
            if new_id not in processed_ids:  # Проверяем, чтобы не было повторов
                dupl_id_list.append(new_id)
                all_duplicates.add(new_id)

        # Помечаем текущий ID как обработанный
        processed_ids.add(current_id)
    # Преобразуем множество all_duplicates в строку для SQL-запроса
    duplicates_string = ", ".join([str(int(x)) if isinstance(x, Decimal) else str(x) for x in all_duplicates])

    return duplicates_string

In [79]:
initial_ids = "113741935"
abc = find_all_duplicates(initial_ids)
print(abc)

7, 1, 4, 5, 3, 9


In [67]:
def find_all_duplicates(initial_ids):
    """
    Функция для поиска всех связанных дубликатов.
    :param initial_ids: Список ID (dupl_id_list), с которых начинается поиск.
    :return: Список всех связанных ID.
    """
    dupl_id_list = list(initial_ids)  # Начальные ID для поиска
    processed_ids = set()  # Храним уже обработанные ID
    all_duplicates = set(dupl_id_list)  # Храним все найденные дубликаты

    while dupl_id_list:
        # Берем текущий ID для поиска
        current_id = dupl_id_list.pop(0)
        if current_id in processed_ids:
            continue  # Пропускаем, если ID уже обработан

        # Выполняем SQL-запрос для поиска всех объявлений, у которых duplicate_id = current_id
        with conn.cursor() as cursor:
            query = """
            SELECT id
            FROM av_test
            WHERE duplicate_id = %s
            """
            cursor.execute(query, (current_id,))
            results = cursor.fetchall()

        # Добавляем найденные ID в список для последующей обработки
        for row in results:
            new_id = row[0]
            if new_id not in processed_ids:  # Проверяем, чтобы не было повторов
                dupl_id_list.append(new_id)
                all_duplicates.add(new_id)

        # Помечаем текущий ID как обработанный
        processed_ids.add(current_id)

    # Преобразуем в список и возвращаем
    return list(all_duplicates)

# Пример использования
initial_ids = [113741935, 113741934]
result = find_all_duplicates(initial_ids)

# Убедимся, что нет объектов Decimal в списке
result = [int(x) if isinstance(x, Decimal) else x for x in result]

print("Все дубликаты:", result)

NameError: name 'Decimal' is not defined

In [64]:
id_to_double_check

[Decimal('113741933'),
 Decimal('113741934'),
 Decimal('113741933'),
 Decimal('113741934')]